In [1]:
#tfrecode로 바꿔보자.
import os
import sys
import random
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET
import datetime

In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string(
    'dataset_name', 'pascalvoc',
    'The name of the dataset to convert.')
tf.app.flags.DEFINE_string(
    'dataset_dir', './data/VOCdevkit/VOC2007/',
    'Directory where the original dataset is stored.')
tf.app.flags.DEFINE_string(
    'output_name', 'pascalvoc',fo
    'Basename used for TFRecords output files.')
tf.app.flags.DEFINE_string(
    'output_dir', './',
    'Output directory where to store TFRecords files.')

In [3]:
VOC_LABELS = {
    'none': (0, 'Background'),
    'aeroplane': (1, 'Vehicle'),
    'bicycle': (2, 'Vehicle'),
    'bird': (3, 'Animal'),
    'boat': (4, 'Vehicle'),
    'bottle': (5, 'Indoor'),
    'bus': (6, 'Vehicle'),
    'car': (7, 'Vehicle'),
    'cat': (8, 'Animal'),
    'chair': (9, 'Indoor'),
    'cow': (10, 'Animal'),
    'diningtable': (11, 'Indoor'),
    'dog': (12, 'Animal'),
    'horse': (13, 'Animal'),
    'motorbike': (14, 'Vehicle'),
    'person': (15, 'Person'),
    'pottedplant': (16, 'Indoor'),
    'sheep': (17, 'Animal'),
    'sofa': (18, 'Indoor'),
    'train': (19, 'Vehicle'),
    'tvmonitor': (20, 'Indoor'),
}

In [4]:
def int64_feature(value):
    """Wrapper for inserting int64 features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def float_feature(value):
    """Wrapper for inserting float features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [5]:
# Original dataset organisation.
DIRECTORY_ANNOTATIONS = 'Annotations/'
DIRECTORY_IMAGES = 'JPEGImages/'

# TFRecords convertion parameters.
RANDOM_SEED = datetime.date.today().year + datetime.date.today().month + datetime.date.today().day 
SAMPLES_PER_FILES = 200

In [39]:
def _process_image(directory, name): # 4번 - 디렉토리와 이미지네임을 가져다가 annotation xml 파일로부터 각종 정보를 리턴 
    """Process a image and annotation file.
    Args:
      filename: string, path to an image file e.g., '/path/to/example.JPG'.
      coder: instance of ImageCoder to provide TensorFlow image coding utils.
    Returns:
      image_buffer: string, JPEG encoding of RGB image.
      height: integer, image height in pixels.
      width: integer, image width in pixels.
    """
    # Read the image file.
    filename = directory + DIRECTORY_IMAGES + name + '.jpg' #파일네임 
    image_data = tf.gfile.FastGFile(filename, 'r').read() #리드 

    # Read the XML annotation file.
    filename = os.path.join(directory, DIRECTORY_ANNOTATIONS, name + '.xml') #엑스엠엘
    tree = ET.parse(filename) #파싱준비
    root = tree.getroot() #루트얻어옴 

    # Image shape.
    size = root.find('size')
    shape = [int(size.find('height').text),
             int(size.find('width').text),
             int(size.find('depth').text)]
    
    # Find annotations.
    bboxes = []
    labels = []
    labels_text = []
    difficult = []
    truncated = []
    for obj in root.findall('object'): #object 다 찾아서 넣음 바운딩박스/라벨(번호)/라벨텍스트(이름)/voc에 있는 특징? 어려운거?/좀짤린거
        label = obj.find('name').text
        labels.append(int(VOC_LABELS[label][0]))
        labels_text.append(label.encode('ascii'))

        if obj.find('difficult'):
            difficult.append(int(obj.find('difficult').text))
        else:
            difficult.append(0)
        if obj.find('truncated'):
            truncated.append(int(obj.find('truncated').text))
        else:
            truncated.append(0)

        bbox = obj.find('bndbox')
        bboxes.append((float(bbox.find('ymin').text) / shape[0],
                       float(bbox.find('xmin').text) / shape[1],
                       float(bbox.find('ymax').text) / shape[0],
                       float(bbox.find('xmax').text) / shape[1]
                       ))
    return image_data, shape, bboxes, labels, labels_text, difficult, truncated

In [28]:
def _convert_to_example(image_data, labels, labels_text, bboxes, shape,
                        difficult, truncated):
    """Build an Example proto for an image example.
    Args:
      image_data: string, JPEG encoding of RGB image;
      labels: list of integers, identifier for the ground truth;
      labels_text: list of strings, human-readable labels;
      bboxes: list of bounding boxes; each box is a list of integers;
          specifying [xmin, ymin, xmax, ymax]. All boxes are assumed to belong
          to the same label as the image label.
      shape: 3 integers, image shapes in pixels.
    Returns:
      Example proto
    """
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    for b in bboxes:
        assert len(b) == 4
        # pylint: disable=expression-not-assigned
        [l.append(point) for l, point in zip([ymin, xmin, ymax, xmax], b)] #l > coordinates list / b > boundingbox index
        #2차원 배열([ymin, xmin, ymax, xmax])에 boundindx box 좌표를 순서대로 넣어주는 것 같다. 
        # ???? 잘 모르곘음 소수점이 막 나오는 리스트가 l이 되긴 하는데... 
        # pylint: enable=expression-not-assigned
        
    image_format = b'JPEG' #이미지 포멧 
    example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': int64_feature(shape[0]),
            'image/width': int64_feature(shape[1]),
            'image/channels': int64_feature(shape[2]),
            'image/shape': int64_feature(shape),
            'image/object/bbox/xmin': float_feature(xmin),
            'image/object/bbox/xmax': float_feature(xmax),
            'image/object/bbox/ymin': float_feature(ymin),
            'image/object/bbox/ymax': float_feature(ymax),
            'image/object/bbox/label': int64_feature(labels),
            'image/object/bbox/label_text': bytes_feature(labels_text),
            'image/object/bbox/difficult': int64_feature(difficult),
            'image/object/bbox/truncated': int64_feature(truncated),
            'image/format': bytes_feature(image_format), #포멧
            'image/encoded': bytes_feature(image_data)})) #실데이터
    return example

In [29]:
def _add_to_tfrecord(dataset_dir, name, tfrecord_writer): #3번 - 데이터 불러와서 tf recode에 써줌 
    """Loads data from image and annotations files and add them to a TFRecord.
    Args:
      dataset_dir: Dataset directory;
      name: Image name to add to the TFRecord;
      tfrecord_writer: The TFRecord writer to use for writing.
    """
    image_data, shape, bboxes, labels, labels_text, difficult, truncated = _process_image(dataset_dir, name) #이미지 어노테이션 데이터 
    example = _convert_to_example(image_data, labels, labels_text, bboxes, shape, difficult, truncated) #받아온 데이터로 example proto만듬
    tfrecord_writer.write(example.SerializeToString()) #시리얼라이즈 해서 써줌 

In [30]:
def _get_output_filename(output_dir, name, idx): #2번 - output filename 만들어줌. 
    return '%s/%s_%03d.tfrecords' % (output_dir, name, idx)

In [31]:
def run(dataset_dir, output_dir, name='voc_train', shuffling=False): #1번 - 데이터 dir, 아웃풋 dir, 이름, 셔플여부 
    """Runs the conversion operation.
    Args:
      dataset_dir: The dataset directory where the dataset is stored.
      output_dir: Output directory.
    """
    if not tf.gfile.Exists(dataset_dir):
        print('data directory 없다. 없으면 못쓰지')
        tf.gfile.MakeDirs(dataset_dir)

    # Dataset filenames, and shuffling. > annotation xml file list
    path = os.path.join(dataset_dir, DIRECTORY_ANNOTATIONS)
    filenames = sorted(os.listdir(path))

    if shuffling:
        random.seed(RANDOM_SEED)
        random.shuffle(filenames)

    # Process dataset files.
    i = 0
    fidx = 0
    while i < len(filenames): # all files
        # Open new TFRecord file.
        tf_filename = _get_output_filename(output_dir, name, fidx) #파일네임 받아옴. 
        with tf.python_io.TFRecordWriter(tf_filename) as tfrecord_writer: #tf recode writer 
            j = 0
            while i < len(filenames) and j < SAMPLES_PER_FILES: #한 tfrecode에 SAMPLES_PER_FILES만큼 넣어줌. 
                sys.stdout.write('\r>> Converting image %d/%d' % (i+1, len(filenames)))
                sys.stdout.flush()

                filename = filenames[i]
                img_name = filename[:-4] #이미지네임 가지고 tfrecode 만들어줌
                _add_to_tfrecord(dataset_dir, img_name, tfrecord_writer)
                i += 1
                j += 1
            fidx += 1 #filename index

    # Finally, write the labels file:
    # labels_to_class_names = dict(zip(range(len(_CLASS_NAMES)), _CLASS_NAMES))
    # dataset_utils.write_label_file(labels_to_class_names, dataset_dir)
    print('\nFinished converting the Pascal VOC dataset!')

In [40]:
run(FLAGS.dataset_dir, FLAGS.output_dir, FLAGS.output_name)

>> Converting image 5011/5011
Finished converting the Pascal VOC dataset!


In [ ]:
predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
def decode_jpeg(self, image_data):
    image = self._sess.run(self._decode_jpeg,
                           feed_dict={self._decode_jpeg_data: image_data})
    assert len(image.shape) == 3
    assert image.shape[2] == 3
    return image
decode_jpeg_data = tf.placeholder(dtype=tf.string)
decode_jpeg = tf.image.decode_jpeg(decode_jpeg_data, channels=3)

#self._png_to_jpeg = tf.image.encode_jpeg(image, format='rgb', quality=100)

##### 1. 이미지 GFast - distortion 포기해야 함. shuffle batch도 아직 어려움 + image height,width 알아야 함
##### 2. 이미지 그대로 uint8로 읽어오기 - 속도가 조금 느려짐. distortion, shuffle 다 가능 
##### 3. 

https://github.com/tensorflow/models/blob/master/inception/inception/data/build_image_data.py
https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/examples/how_tos/reading_data/fully_connected_reader.py

In [51]:
H, W, B = 24, 78, 9
anchor_shapes = np.reshape(
  [np.array(
      [[  36.,  37.], [ 366., 174.], [ 115.,  59.],
       [ 162.,  87.], [  38.,  90.], [ 258., 173.],
       [ 224., 108.], [  78., 170.], [  72.,  43.]])] * H * W,
  (H, W, B, 2)
)

In [52]:
print(anchor_shapes.shape)

IMAGE_WIDTH           = 1242
IMAGE_HEIGHT          = 375

(24, 78, 9, 2)


In [46]:
center_x = np.reshape(
  np.transpose(
      np.reshape(
          np.array([np.arange(1, W+1)*float(IMAGE_WIDTH)/(W+1)]*H*B), 
          (B, H, W)
      ),
      (1, 2, 0)
  ),
  (H, W, B, 1)
)
center_y = np.reshape(
  np.transpose(
      np.reshape(
          np.array([np.arange(1, H+1)*float(IMAGE_HEIGHT)/(H+1)]*W*B),
          (B, W, H)
      ),
      (2, 1, 0)
  ),
  (H, W, B, 1)
)
anchors = np.reshape(
  np.concatenate((center_x, center_y, anchor_shapes), axis=3),
  (-1, 4)
)


In [54]:
len(anchors[:,1])

16848

In [55]:
line = "a d w e g s"

In [56]:
line

'a d w e g s'

In [57]:
line.strip()

'a d w e g s'

In [58]:
line.split(' ' )

['a', 'd', 'w', 'e', 'g', 's']

In [59]:
line.strip().split(' ' )

['a', 'd', 'w', 'e', 'g', 's']

In [70]:
dets_per_typ = {} 

In [61]:
type(dets_per_type)

dict

In [75]:
dets_per_typ['z'] = [{'1':'11'}]
dets_per_typ['z'].append({'22':'222'})

In [76]:
dets_per_typ

{'b': [{'1': '11'}, {'22': '222'}], 'z': [{'1': '11'}, {'22': '222'}]}

In [78]:
for error_type, dets in dets_per_typ.items():
    print(error_type)
    print(dets)

b
[{'1': '11'}, {'22': '222'}]
z
[{'1': '11'}, {'22': '222'}]
